# Toggling Objective — IPOPT Resource Analysis (Julia)

This notebook copies the problem setup and IPOPT solve code from your original notebook, and adds log capture, parsing, and plotting for resource analysis.

## Copied problem setup & solve code

In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.develop(path="../../QuantumCollocation.jl")
# Pkg.develop(path="../../Piccolissimo.jl")
using PiccoloQuantumObjects
using QuantumCollocation
using ForwardDiff
using LinearAlgebra
using Plots
using SparseArrays
using Statistics
using CairoMakie
using NamedTrajectories
using TrajectoryIndexingUtils
using Random
using CSV
using Measures: mm
using Random
using FilePathsBase, FileIO

In [ ]:
n_drives = sys.n_drives
n_guesses = 3
def_seeds = []
add_seeds = []
mult_seeds = []
both_seeds = []

for i in 1:n_guesses
    Random.seed!(i*124)
    a_bounds = fill(1.0, n_drives)
    da_bounds = fill((3+i)π*Δt/T, n_drives)
    dda_bounds = fill(1.0, n_drives)
    control_bounds = (a_bounds, da_bounds, dda_bounds)
    def_traj = initialize_trajectory(
                    U_goal,
                    T,
                    Δt,
                    n_drives,
                    control_bounds;
                    system=sys
                )
    push!(def_seeds, def_traj)

    add_traj = initialize_trajectory(
                    U_goal,
                    T,
                    Δt,
                    n_drives,
                    control_bounds;
                    system=sys
                )

    push!(add_seeds, add_traj)

    mult_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                system=sys
            )

    push!(mult_seeds, mult_traj)

    both_traj = initialize_trajectory(
                U_goal,
                T,
                Δt,
                n_drives,
                control_bounds;
                system=sys
            )

    push!(both_seeds, both_traj)
end

In [ ]:
sweep_rob_loss_λ = exp.(range(log(.1), log(1), length=6))
n_seeds = n_guesses
n_lambdas = length(sweep_rob_loss_λ)

default_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
add_probs = Matrix{Any}(undef, n_seeds, n_lambdas)
mult_probs = Matrix{Any}(undef, n_seeds, n_lambdas)

Hₑ_add = a -> PAULIS.X
X_drive = sys.H.H_drives[1]
Hₑ_mult = a -> a[1] * X_drive


for i in 1:n_seeds
    for (λ_idx, λ) in enumerate(sweep_rob_loss_λ) 

        # Add problem
        add_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt;
            init_trajectory=deepcopy(add_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=Hₑ_add,
            Q_t=λ
        )
        solve!(add_prob; max_iter=50, print_level=5)
        add_probs[i, λ_idx] = add_prob

        # default
        defaults = UnitarySmoothPulseProblem(sys, U_goal, T, Δt; init_trajectory=deepcopy(def_seeds[i]))
        solve!(defaults; max_iter=50, print_level=5)
        default_probs[i,λ_idx] = defaults

        # mult
        mult_prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt;
            init_trajectory=deepcopy(mult_seeds[i]),
            piccolo_options=piccolo_opts,
            activate_rob_loss=true,
            H_err=Hₑ_mult,
            Q_t=λ
        )
        solve!(mult_prob; max_iter=50, print_level=5)
        mult_probs[i, λ_idx] = mult_prob
    end
end


## IPOPT log capture helper

In [ ]:
function capture_ipopt_log(f::Function)
    buf = IOBuffer()
    result = nothing
    Base.redirect_stdout(buf) do
        Base.redirect_stderr(buf) do
            result = f()
        end
    end
    return (String(take!(buf)), result)
end

## IPOPT log parser

In [ ]:
using DataFrames

const RE_ITER  = r"(Number of Iterations)\s*[:=]\s*(\d+)"
const RE_WALL  = r"(Total wall clock secs|Total time \(wall\))\s*[:=]\s*([0-9]*\.?[0-9]+)"
const RE_CPU   = r"(Total CPU secs|Total time \(CPU\))\s*[:=]\s*([0-9]*\.?[0-9]+)"
const RE_OBJ   = r"(Objective|Final Objective)\s*[:=]\s*([\-+]?[0-9]*\.?[0-9]+([eE][\-+]?\d+)?)"
const RE_STAT  = r"(EXIT:|Optimal Solution Found|Stop at iteration|Solved To Acceptable Level|Converged|Infeasible|Maximum Number of Iterations Exceeded)"

function parse_ipopt_log(blob::AbstractString)
    iterations, wall_time_sec, cpu_time_sec, objective, status = missing, missing, missing, missing, missing
    if m = match(RE_ITER, blob); m !== nothing; iterations = parse(Int, m.captures[2]); end
    if m = match(RE_WALL, blob); m !== nothing; wall_time_sec = parse(Float64, m.captures[2]); end
    if m = match(RE_CPU, blob); m !== nothing; cpu_time_sec = parse(Float64, m.captures[2]); end
    objs = collect(eachmatch(RE_OBJ, blob))
    if !isempty(objs); objective = parse(Float64, objs[end].captures[2]); end
    stats = collect(eachmatch(RE_STAT, blob))
    if !isempty(stats); status = stats[end].match; end
    return (; iterations, wall_time_sec, cpu_time_sec, objective, status)
end